# Field, Goldsmith, & Habing Multi-Phase ISM

Figure 1.10-1.12 from Chapter 1 of *Interstellar and Intergalactic Medium* by Ryden & Pogge, 2021, 
Cambridge University Press.

This notebook creates figures illustrating the Field, Goldsmith, and Habing (FGH) multi-phase interstellar
medium model [Field, Goldsmith, & Habing 1969, ApJ, 155, L149](https://ui.adsabs.harvard.edu/abs/1969ApJ...155L.149F/abstract)

There are 3 figures
 * Figure 1.10 - FGH Cooling function $\Lambda(T)$
 * Figure 1.11 - Equilibrium density $n_{eq}(T)$
 * Figure 1.12 - Pressure $P$ vs density $n_{eq}$

In [ ]:
%matplotlib inline

import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import warnings
warnings.filterwarnings('ignore',category=UserWarning, append=True)

## Standard Plot Format

Setup the standard plotting format and make the plot. Fonts and resolution adopted follow CUP style.

In [ ]:
# graphic aspect ratio = width/height

aspect = 4.0/3.0

# Text width in inches - don't change, this is defined by the print layout

textWidth = 6.0 # inches

# output format and resolution

figFmt = 'png'
dpi = 600

# Graphic dimensions 

plotWidth = dpi*textWidth
plotHeight = plotWidth/aspect
axisFontSize = 14
labelFontSize = 10
lwidth = 0.5
axisPad = 5
wInches = textWidth 
hInches = wInches/aspect

# LaTeX is used throughout for markup of symbols, Times-Roman serif font

plt.rc('text', usetex=True)
plt.rc('font', **{'family':'serif','serif':['Times-Roman'],'weight':'bold','size':'16'})

# Font and line weight defaults for axes

matplotlib.rc('axes',linewidth=lwidth)
matplotlib.rcParams.update({'font.size':axisFontSize})

# axis and label padding

plt.rcParams['xtick.major.pad'] = f'{axisPad}'
plt.rcParams['ytick.major.pad'] = f'{axisPad}'
plt.rcParams['axes.labelpad'] = f'{axisPad}'

## FGH Model Calculation

The basic parameters for these models are set in the code section below:

 * Ionization fraction: $x_e$=0.001
 * Temperature Range: $T_e$=10 to 20000K (logratithmic)
 * gain factor: $G$=20, defined such that $n_{eq}$=G/$\Lambda$ ($\Lambda$ is the total cooling rate)

The model assumes three sources of collisional cooling using these scaling relations:

HI Lyman-$\alpha$ Cooling (Eqn 1.38):
\begin{equation}
  \frac{\Lambda^{e}_{Ly\alpha}}{10^{-27}{\rm erg\,cm^3\,s^{-1}}} \approx 
  6\times10^{5} \left(\frac{x}{0.001}\right)
  \left(\frac{T}{10^{4}K}\right)^{-1/2}
  \exp\left(-\frac{1.18\times10^{5}K}{T}\right)
\end{equation}
 
Carbon (CII) Cooling (Eqn 1.35) electron collisional term:
\begin{equation}
  \frac{\Lambda^{e}_{CII}}{10^{-27}{\rm erg\,cm^3\,s^{-1}}} \approx 
  3.1 \left(\frac{x}{0.001}\right)
  \left(\frac{T}{100K}\right)^{-0.5}
  \exp\left(-\frac{91.2K}{T}\right)
\end{equation}
and H collisional term:
\begin{equation}
  \frac{\Lambda^{H}_{CII}}{10^{-27}{\rm erg\,cm^3\,s^{-1}}} \approx 
  5.2\left(\frac{T}{100K}\right)^{0.13}
  \exp\left(-\frac{91.2K}{T}\right)
\end{equation}

Oxygen (OI) Cooling:
\begin{equation}
   \frac{\Lambda^{H}_{OI}}{10^{-27}{\rm erg\,cm^3\,s^{-1}}} \approx 
   4.1\left(\frac{T}{100K}\right)^{0.42}
   \exp\left(-\frac{228K}{T}\right)
\end{equation}

We compute total cooling ($\Lambda=\Lambda_{Ly\alpha}+\Lambda_{CII}+\Lambda_{OII}$), equilibrium density
($n_{eq}$), and pressure ($P=n_{eq}kT$) as a function of logarithmic steps in temperature.

We have adopted the Lodders (2010) abundances for C and O, as used in the ISM/IGM book 
(see Chapter 1, Table 1.2).

In [ ]:
xe = 0.001
minT = 10.0
maxT = 20000.
gain = 20.0

# Boltzmann Constant (CODATA 2018)

k = 1.380649e-16 # erg K^-1

minLogT = math.log10(minT)
maxLogT = math.log10(maxT)

logT = np.linspace(minLogT,maxLogT,num=1001)
T = 10.0**logT

xfac = xe/0.001
TH = 118000.0 # hydrogen excitation temperature in K
TC = 91.2     # carbon excitation temperature in K
TO = 228.0    # oxygen excitation temperature in K

# Lyman-alpha cooling

coolLya = 6.0e5*(xfac/np.sqrt(T/1.0e4))*np.exp(-TH/T)

# Carbon cooling

coolC = 3.1*(xfac/np.sqrt(T/100.0))*np.exp(-TC/T) + 5.2*((T/100.0)**0.13)*np.exp(-TC/T)

# Oxygen cooling

coolO = 4.1*((T/100.0)**0.42)*np.exp(-TO/T)

# Total cooling

coolTot = (coolLya + coolC + coolO)

# equilibrium density

neq = gain/coolTot

# pressure

P = neq*k*T


## FGH Cooling Function - Figure 1.10

Plot the cooling function $\Lambda(T)$ vs $T$ including the curves for the individual contributions


In [ ]:
plotFile = f'Fig1_10.{figFmt}'

fig,ax = plt.subplots()
fig.set_dpi(dpi)
fig.set_size_inches(wInches,hInches,forward=True)

ax.tick_params('both',length=6,width=lwidth,which='major',direction='in',top='on',right='on')
ax.tick_params('both',length=3,width=lwidth,which='minor',direction='in',top='on',right='on')

# Limits

minCool = 1.0e-30 # erg cm^3 s^-1
maxCool = 1.0e-24

# Labels

xLabel = r'Temperature [K]'
yLabel = r'$\Lambda$ [erg cm$^3$ s$^{-1}$]'

plt.xlim(minT,maxT)
ax.set_xscale('log')
ax.set_xticks([10,100,1000,1.0e4])
ax.set_xticklabels(['10','100','1000','10$^{4}$'])
plt.xlabel(xLabel)

plt.ylim(minCool,maxCool)
ax.set_yscale('log')
ax.set_yticks([1.0E-30,1.0E-29,1.0E-28,1.0E-27,1.0e-26,1.0e-25,1.0e-24])
ax.set_yticklabels(['$10^{-30}$','10$^{-29}$','10$^{-28}$','10$^{-27}$','10$^{-26}$','10$^{-25}$','10$^{-24}$'])
plt.ylabel(yLabel)

# Plot the total and individual cooling functions

plt.plot(T,1.0e-27*coolTot,'-',color='black',lw=2,zorder=10)
plt.plot(T,1.0e-27*coolLya,'--',color='black',lw=1,zorder=10)
plt.plot(T,1.0e-27*coolC,':',color='black',lw=1,zorder=10)
plt.plot(T,1.0e-27*coolO,'-.',color='black',lw=1,zorder=10)

# label components

lfs = np.rint(1.2*axisFontSize)
plt.text(1000.0,1.7e-26,'Total',fontsize=lfs,rotation=10.0,ha='center',va='bottom')
plt.text(80.0,1.0e-28,r'$[\textsc{O\,i}]\,\lambda$63$\mu m$',fontsize=lfs)
plt.text(3000.0,3.5e-27,r'$[\textsc{C\,ii}]\,\lambda$158$\mu m$',fontsize=lfs,rotation=3.0,ha='center')
plt.text(5400.0,1.0e-28,r'Ly$\alpha$',fontsize=lfs,ha='center')

# make the figure

plt.plot()
plt.savefig(plotFile,bbox_inches='tight',facecolor='white')

## FGH equilibrium density - Figure 1.11

Plot the equlibrium density function $n_{eq}$ vs $T$ for the FGH model.


In [ ]:
plotFile = f'Fig1_11.{figFmt}'

fig,ax = plt.subplots()
fig.set_dpi(dpi)
fig.set_size_inches(wInches,hInches,forward=True)

ax.tick_params('both',length=6,width=lwidth,which='major',direction='in',top='on',right='on')
ax.tick_params('both',length=3,width=lwidth,which='minor',direction='in',top='on',right='on')

# Limits

minNe = 0.01     # cm^{-3}
maxNe = 20000.0 

# Labels

xLabel = r'Temperature [K]'
yLabel = r'$n$ [cm$^{-3}$]'

plt.xlim(minT,maxT)
ax.set_xscale('log')
ax.set_xticks([10,100,1000,1.0e4])
ax.set_xticklabels(['10','100','1000','10$^{4}$'])
plt.xlabel(xLabel)

plt.ylim(minNe,maxNe)
ax.set_yscale('log')
ax.set_yticks([0.01,0.1,1.0,10.,100.,1e3,1e4])
ax.set_yticklabels(['0.01','0.1','1','10','100','1000','10$^{4}$'])
plt.ylabel(yLabel)

# Plot neq vs T

plt.plot(T,neq,'-',color='black',lw=2,zorder=10)
plt.fill_between(T,neq,maxNe,facecolor="#eaeaea")

# label regions above and below

lfs = np.rint(1.2*axisFontSize)
plt.text(200.0,0.1,'Net heating',fontsize=lfs,ha='center',zorder=10)
plt.text(1000.0,20.0,'Net cooling',fontsize=lfs,ha='center',zorder=10)

# make the figure

plt.plot()
plt.savefig(plotFile,bbox_inches='tight',facecolor='white')

## FGH pressure vs density - Figure 1.12

Plot the equlibrium pressure vs density for the FGH model.


In [ ]:
plotFile = f'Fig1_12.{figFmt}'

fig,ax = plt.subplots()
fig.set_dpi(dpi)
fig.set_size_inches(wInches,hInches,forward=True)

plt.tick_params('both',length=6,width=lwidth,which='major',direction='in',top='on',right='on')
plt.tick_params('both',length=3,width=lwidth,which='minor',direction='in',top='on',right='on')

# Limits

minNe = 0.02     # cm^{-3}
maxNe = 10000.0 

minP = 4.0e-14   # dyne cm^-2
maxP = 1.0e-11

# Labels

xLabel = r'$n$ [cm$^{-3}$]'
yLabel = r'$P$ [dyne cm$^{-2}$]'

plt.xlim(minNe,maxNe)
plt.xscale('log')
ax.set_xticks([0.1,1.0,10.,1.0e2,1.0e3,1.0e4])
ax.set_xticklabels(['0.1','1.0','10','100','1000','10$^4$'])
plt.xlabel(xLabel)

plt.ylim(minP,maxP)
ax.set_yscale('log')
ax.set_yticks([1.0e-13,1.0e-12,1.0e-11])
ax.set_yticklabels(['10$^{-13}$','10$^{-12}$','10$^{-11}$'])
plt.ylabel(yLabel)

# plot the n-P curve

plt.plot(neq,P,'-',color='black',lw=2,zorder=10)
plt.fill_between(neq,P,maxP,facecolor="#eaeaea")

# FGH stability region - need to measure these by hand from the plots

fghUpper = 6.72e-13
fghLower = 7.35e-14
plt.plot([minNe,maxNe],[fghLower,fghLower],color='black',ls='--',lw=0.5)
plt.plot([minNe,maxNe],[fghUpper,fghUpper],color='black',ls='--',lw=0.5)

# F, G, and H points along P=2e-13 dyne/cm^2, also "measured" by hand for plotting purposes

yFGH = 2.0e-13
xF = 0.0985
xG = 1.14 
xH = 52.6

lfs = np.rint(1.2*axisFontSize)

plt.plot(xF,yFGH,color='black',marker='o',ms=8,mfc='black')
plt.text(1.4*xF,yFGH,'F',fontsize=lfs,va='center',zorder=10)
plt.plot(xG,yFGH,color='black',marker='o',ms=8,mfc='black')
plt.text(1.4*xG,yFGH,'G',fontsize=lfs,va='center',zorder=10)
plt.plot(xH,yFGH,color='black',marker='o',ms=8,mfc='black')
plt.text(1.4*xH,yFGH,'H',fontsize=lfs,va='center',zorder=10)

plt.text(10.0,2.0e-12,'Net cooling',fontsize=lfs,ha='center',va='center',zorder=10)
plt.text(1300.0,yFGH,'Net heating',fontsize=lfs,ha='center',va='center',zorder=10)

# make the figure

plt.plot()
plt.savefig(plotFile,bbox_inches='tight',facecolor='white')